In [2]:
import os
import pickle  # nosec
import subprocess  # nosec
from collections import defaultdict

import click
import datasets
import pandas as pd
from datasets import Features
from datasets import Sequence
from datasets import Value

from text_dedup.ann_unisim import main as unisim_main
from text_dedup.minhash import main as minhash_main
from text_dedup.simhash import main as simhash_main
from text_dedup.utils import IOArgs
from text_dedup.utils import MetaArgs
from text_dedup.utils import MinHashArgs
from text_dedup.utils import SimHashArgs
from text_dedup.utils import Timer
from text_dedup.utils import UnionFind
from text_dedup.utils import UniSimArgs

NUM_PROC = os.cpu_count()

In [4]:
ds = datasets.load_from_disk("temp_inp_ds")
truth = ds.map(
    lambda x, idx: {"core_id": x["core_id"], "id": idx, "duplicates": x["labelled_duplicates"]},
    remove_columns=ds.column_names,
    with_indices=True,
    num_proc=NUM_PROC,
    features=Features(
        {
            "core_id": Value("string"),
            "id": Value("int64"),
            "duplicates": Sequence(Value("string")),
        }
    ),
)

Map (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [6]:
import polars as pl


df = pl.read_parquet("/home/wayne/kioxia/github/text-dedup/temp_inp_paruqet/data.parquet")

In [8]:
len(df)

100000

In [26]:
df['labelled_duplicates'].map_elements(lambda s: len(s)) > 0

/tmp/ipykernel_21013/3664851347.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df['labelled_duplicates'].map_elements(lambda s: len(s)) > 0


labelled_duplicates
bool
true
true
true
true
true
…
false
false
false
